In [11]:
# coding=utf-8
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

options = FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)
# driver = webdriver.Firefox()

first = True
def add_info(url):
	global first
	driver.get(url)
	wait = WebDriverWait(driver, 20)
	if "gehalt.de" in url:
		wait.until(lambda driver: driver.current_url != url)
	link = driver.current_url
	if "stepstone.de" not in link:
		return(link, 
				"Nicht stepstone",
				"Nicht stepstone",
				"Nicht stepstone",
				"Nicht stepstone",
				"Nicht stepstone",
				"Nicht stepstone",)
	# print(link)
	if first:
		wait.until(lambda driver: len(driver.find_elements(By.CSS_SELECTOR, "#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button")) > 0)
		driver.find_element(By.CSS_SELECTOR, "#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button").click()
		driver.find_element(By.CSS_SELECTOR, ".secondary-button.ccmgt_reject_button").click()
		first = False

	if len(driver.find_elements(By.CLASS_NAME, "listing-content-provider-hw0nf9")) > 0:
		driver.find_element(By.CLASS_NAME, "listing-content-provider-hw0nf9").click()
	
	response = driver.page_source
	soup_object = BeautifulSoup(response)
	if len(soup_object.find_all(attrs={"data-genesis-element":"ALERT_CONTENT"})) > 0:
		return(link, 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar")
	# Teilzeit und Remote
	if len(soup_object.find_all(class_="at-listing__list-icons_work-type")) > 0:
		work_type = soup_object.find(class_="at-listing__list-icons_work-type").find(attrs={"data-genesis-element":"TEXT"})
		# # print(soup_object)
		# print(work_type)
		teilzeit_remote = work_type.text
	else:
		teilzeit_remote = "-"

	base = soup_object.find_all("div", class_="sc-EHOje jWEhHL")

	texts = ["", "", "", "", ""]

	for i in range(len(base)):
		area = base[i].find_all(["p", "li"])
		for element in area:
			texts[i] += element.text + "\n"
	
	if (teilzeit_remote == "-") & (texts == ["", "", "", "", ""]):
		errors = pd.read_pickle("../data/errors.pkl")
		idx = len(errors)
		errors.loc[idx] = link, str(soup_object)
		errors.to_pickle("../data/errors.pkl")

	return link, teilzeit_remote, texts[0], texts[1], texts[2], texts[3], texts[4]


In [7]:
jobs = pd.read_pickle("../data/jobs.pkl")
jobs1 = jobs[(jobs["Teilzeit_Remote"] == "-") & (jobs["Introduction"] == "")]
jobs1


,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,Description,Profile,We_offer,Contact
7504,Business Data Analyst (m/w/d),Springer Medizin Verlag GmbH,"Berlin, Heidelberg",3710,4839,57735323,https://www.stepstone.de/stellenangebote--busi...,2023-09-21,-,,,,,
7506,Data Engineer / Analyst (m/f/x),ZEISS,München,,,57736019,https://www.stepstone.de/stellenangebote--data...,2023-09-21,-,,,,,
7509,Controller* Financial Planning & Analysis,NEW YORKER,Braunschweig,,,57721856,https://www.stepstone.de/stellenangebote--cont...,2023-09-22,-,,,,,
7510,Teamleader (f/m/d) Business Analysis,S.OLIVER GROUP,Rottendorf,3790,6129,57725421,https://www.stepstone.de/stellenangebote--team...,2023-09-22,-,,,,,
7513,EMC Analysis & Test Engineer (m/f/d),OHB System AG,Bremen,4677,6290,57725817,https://www.stepstone.de/stellenangebote--emc-...,2023-09-22,-,,,,,
7514,Praktikant im Business Analysis & Reporting (m...,Würth Elektronik eiSos GmbH & Co. KG,Waldenburg,,,57716233,https://www.stepstone.de/stellenangebote--prak...,2023-09-22,-,,,,,
7515,Market Analysis and Sales Planning Manager DAC...,Rodenstock GmbH,München,5081,6774,57679012,https://www.stepstone.de/stellenangebote--mark...,2023-09-22,-,,,,,
7517,Senior Manager Liner Shipping Controlling – Co...,Hapag-Lloyd AG,Hamburg,5726,7500,57702499,https://www.stepstone.de/stellenangebote--seni...,2023-09-22,-,,,,,
7520,Senior Data Engineer (m/w/d) Customer Data,Mercedes-Benz Tech Innovation,Ulm,,,57735080,https://www.stepstone.de/stellenangebote--seni...,2023-09-22,-,,,,,
7523,Data Engineer,Klarna GmbH,"BERLIN, GERMANY",,,57736634,https://www.stepstone.de/stellenangebote--data...,2023-09-22,-,,,,,


In [12]:
indexes = jobs1.index
for i in indexes:
	jobs.loc[i, ["Link", "Teilzeit_Remote", "Introduction", "Description", "Profile", "We_offer", "Contact"]] = add_info(jobs1.loc[i, "Link"])
	# jobs.to_pickle("../data/jobs.pkl")
driver.quit()

In [13]:
jobs1 = jobs[(jobs["Teilzeit_Remote"] == "-") & (jobs["Introduction"] == "")]
jobs1

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,Description,Profile,We_offer,Contact


In [14]:
errors = pd.read_pickle("../data/errors.pkl")
errors

,Link,Soup


In [15]:
jobs.to_pickle("../data/jobs.pkl")